In [2]:
import tensorflow as tf
from data_loader import DataLoader

ds_train, ds_test, ds_info = DataLoader('MNIST', batch_size=128)

2022-03-24 23:54:47.072206: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-24 23:54:47.072781: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-24 23:54:47.077422: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


2022-03-24 23:54:52.314675: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-24 23:54:52.316647: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2808000000 Hz


469/469 [==============================] - 73s 154ms/step - loss: 0.8564 - sparse_categorical_accuracy: 0.7296 - val_loss: 0.1063 - val_sparse_categorical_accuracy: 0.9680
Epoch 2/6
469/469 [==============================] - 53s 113ms/step - loss: 0.1035 - sparse_categorical_accuracy: 0.9700 - val_loss: 0.0714 - val_sparse_categorical_accuracy: 0.9775
Epoch 3/6
469/469 [==============================] - 53s 113ms/step - loss: 0.0677 - sparse_categorical_accuracy: 0.9803 - val_loss: 0.0601 - val_sparse_categorical_accuracy: 0.9811
Epoch 4/6
469/469 [==============================] - 58s 123ms/step - loss: 0.0499 - sparse_categorical_accuracy: 0.9841 - val_loss: 0.0479 - val_sparse_categorical_accuracy: 0.9841
Epoch 5/6
469/469 [==============================] - 55s 118ms/step - loss: 0.0402 - sparse_categorical_accuracy: 0.9872 - val_loss: 0.0443 - val_sparse_categorical_accuracy: 0.9869
Epoch 6/6
469/469 [==============================] - 56s 119ms/step - loss: 0.0305 - sparse_categori

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                73744     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [8]:
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_____________________________________________________

In [17]:
%load_ext autoreload
%autoreload 2

import node

input = tf.keras.Input(shape=(28, 28, 1))

layer1 = node.Convolution2D(32, activation='relu')
layer2 = node.DenseLayer(128, activation='relu')

last_active_layer = node.MaxPool2D()(layer1.create_node(dataset_size=(28, 28))(input))
# last_active_layer = layer1.create_node(dataset_size=(28, 28))(input)

# last_active_layer = layer2(last_active_layer)
flat_layer = tf.keras.layers.Flatten()(last_active_layer)
last_active_layer = layer2.create_node(dataset_size=(28, 28))(flat_layer)

output = tf.keras.layers.Dense(units=10, activation='softmax')(last_active_layer)
model2 = tf.keras.Model(input, output)

model2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

# history = model2.fit(
#     ds_train,
#     epochs=6,
#     validation_data=ds_test,
# )
      
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(28, 28)),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(10)
# ])
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(0.001),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
# )

# model.fit(
#     ds_train,
#     epochs=6,
#     validation_data=ds_test,
# )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


TypeError: Inputs to a layer should be tensors. Got: <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd12a7421f0>

In [ ]:
history = model2.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)